# NLP Homework #2 (Aleksandr Kariakin, Nikita Ivlev, Lev Leontev)

### Generate new sentences using Markovify and Shakespeare plays dataset

**Import the needed libraries**

In [83]:
!pip3 install markovify
!pip3 install numpy

In [84]:
import markovify
import numpy as np

Read the dataset of all Shakespeare plays

In [85]:
with open("shakespeare.txt") as f:
    text = f.read()

Create a function to build the Markov chain model and generate a text with a parameter defining on how many previous words our Markov chain depends 

In [86]:
def generate_sentences(state_size):
    text_model = markovify.Text(text, state_size=state_size)
    return [text_model.make_sentence(tries=100000) for _ in range(100)]

Print five randomly-generated sentences with a Markov chain that depends on 1 words:

In [87]:
sentences_1 = generate_sentences(state_size=1)
print(*sentences_1[:5], sep="\n")

I perceived a jig, you heard too much, methinks, is my face, But bear-like I am forbid; Or else, To speak in these are, with the thing of enemies.
Prince, and loathsome beggar.
I let out of his eyes best to Henry Guildford, This is the little wench, as thou cross it, his wife's a dish that the speediest bring away as much are greater than false of that do as haste and having such interchange of this gear.
As when one must not need; we'll tell truth; Nor is my lord; a casement and so superfluous breath?
Therefore these boys of fortune's malice, For I am: More mild, Than hate thee; in their mothers, maids, full of law, Whereof by a piece of fixed star hath here lies murdered.


Print five randomly-generated sentences with a Markov chain that depends on 2 words:

In [88]:
sentences_2 = generate_sentences(state_size=2)
print(*sentences_2[:5], sep="\n")

Hence, Mardian, And bring me word to-morrow, By one whom she did make no noise- in such manner that it pleases you to live is torment, and then on Romeo cries, And then to horse again; Go, go, up to the rebels, and by my sword.
I would not force in eyes That I, forsooth, in love; yea, he loveth.
Brother, you have good witness of my queen!
As rich shall Romeo's by his worth, They say, she hath not such a violence, the King- Mine and your eyes may pierce I cannot find her.
And away with her sorrow, mesh'd upon her faith!


## Evaluating the quality

To evaluate the quality of the model, we can use A/B testing with English native speakers. We should give each study participant a pair of sentences by our model and from our dataset, without telling them which one is which, and suggest choosing the one that they think is AI-generated.

In order to evaluate the quality automatically, we could use several approaches.

### Spelling and grammar checkers

Import and initialize LanguageTool (a free and open source library for checking texts):

In [89]:
!pip3 install language-tool-python

In [90]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-GB')  # use a local server (automatically set up), language English

Test checking sentences with LanguageTool

In [91]:
for i in range(5):
    print(tool.check(sentences_2[i]))

[Match({'ruleId': 'MORFOLOGIK_RULE_EN_GB', 'message': 'Possible spelling mistake found.', 'replacements': ['Marian', 'Martian', 'Marxian', 'Mardi an'], 'offsetInContext': 7, 'context': 'Hence, Mardian, And bring me word to-morrow, By one wh...', 'offset': 7, 'errorLength': 7, 'category': 'TYPOS', 'ruleIssueType': 'misspelling', 'sentence': 'Hence, Mardian, And bring me word to-morrow, By one whom she did make no noise- in such manner that it pleases you to live is torment, and then on Romeo cries, And then to horse again; Go, go, up to the rebels, and by my sword.'}), Match({'ruleId': 'EN_SPLIT_WORDS_HYPHEN', 'message': 'This word seems to be formatted incorrectly. Consider fixing the spacing or removing the hyphen completely.', 'replacements': ['noise-in', 'noise - in'], 'offsetInContext': 43, 'context': '... to-morrow, By one whom she did make no noise- in such manner that it pleases you to live...', 'offset': 73, 'errorLength': 9, 'category': 'TYPOS', 'ruleIssueType': 'misspelling',

Create a metric based on error suggestions per each word on average

In [92]:
def language_tool_metric(sentence):
    count_words = len(sentence.split(" "))
    count_errors = len(tool.check(sentence))
    return count_errors / count_words

Evaluate the average metric value on our generated sentences with the Markov chain of state size 1:

In [93]:
average_language_tool_metric = np.average([language_tool_metric(sentence) for sentence in sentences_1])
print("On average", average_language_tool_metric, "mistakes found per word of text")

On average 0.03919267994104257 mistakes found per word of text


Evaluate the average metric value on our generated sentences with the Markov chain of state size 2:

In [94]:
average_language_tool_metric = np.average([language_tool_metric(sentence) for sentence in sentences_2])
print("On average", average_language_tool_metric, "mistakes found per word of text")

On average 0.03314856663202676 mistakes found per word of text


On our dataset it unfortunately does not produce stable results, because Shakespeare mostly didn't make spelling mistakes, and also he was using an older version of English.

## Other approaches

Possible approaches to automatically check the quality of the generated texts are using LLMs such as Chat GPT instead of human evaluation. Additionally, we could analyze n-grams with the size more than we have chosen in markov chain parameter and check the content quality not only of phrases. 